In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import numpy as np
import os

In [2]:
import pandas as pd

In [43]:
# function for getting full transcript from vtt
import webvtt

def get_transcript(filename) -> str:
    vtt = webvtt.read(filename)
    transcript = ""

    lines = []
    for line in vtt:
        lines.extend(line.text.strip().splitlines())

    previous = None
    for line in lines:
        if line == previous:
           continue
        transcript += " " + line
        previous = line

    # print(transcript)
    return(transcript)

In [13]:
import os
count = 0
for path in os.scandir('subs'):
    count += 1
    print(path.name)

print(count)

--6KU6oa9-Y.en.vtt
-0qg0tkKJWY.en.vtt
-10vg90ZIy8.en.vtt
-1jOm_dfs0w.en.vtt
-2_0OMiiaQI.en.vtt
-3NmMx6U0l0.en.vtt
-4KDFaBHEvw.en.vtt
-6ulcXYFyyQ.en.vtt
-72P_EFphSc.en.vtt
-827QmRDjUA.en.vtt
-9sH5CvLe_s.en.vtt
-aeSRx3J91U.en.vtt
-aQ2E0mlRQI.en.vtt
-atrfTvXEzQ.en.vtt
-avpx8UTakI.en-CA.vtt
-AW_xCVvqXU.en.vtt
-AYxOuSFoVc.en.vtt
-BF9LUGX95k.en.vtt
-BsrzO7aXNs.en.vtt
-cS8VaQno3g.en.vtt
-DTwHKQ2LHs.en.vtt
-dWuivWoScY.en.vtt
-f0bM1ao1jU.en.vtt
-fPJCEHumvE.en.vtt
-fq75Ax_kL0.en.vtt
-fs8gkiap6U.en.vtt
-gZIwtSfizM.en.vtt
-HFwok9SlQQ.en.vtt
-IzgyM1r1y8.en-fr.vtt
-JD33_CLrxc.en.vtt
-jtALmT--rQ.en-1oAFzPNiuC0.vtt
-l7MM9yoxII.en-GB.vtt
-LuqLb7c6s4.en.vtt
-LvMHRT6H24.en.vtt
-mgKxRWe9bY.en.vtt
-MKcgWN1zWM.en.vtt
-nNiULl5_2k.en.vtt
-oKLURMqEB8.en.vtt
-OqrZG-EBaQ.en.vtt
-ox6AoQB1Wo.en.vtt
-O_DMyHdq_M.en.vtt
-q5QJW8q7qc.en.vtt
-qHHHUUeqDk.en.vtt
-RIfYNZrwco.en.vtt
-Rygh1YB42Y.en-nP7-2PuUl7o.vtt
-s1ZxboxcF8.en.vtt
-ST6ychuwTM.en.vtt
-UxRykHXN_0.en.vtt
-VHVLtnpSJQ.en.vtt
-vIaXyQoLH8.en.vtt
-wYHDjU6uuE.en.vt

In [36]:
df_source = pd.read_csv('youtube_audit_dataset_unique.csv')
num_debunk = 0
num_neutral = 0
num_promote = 0
file_list = os.scandir('subs')

filename_list = []
for file in file_list:
    filename_length = len(file.name)
    # temp = str(file.name[1:(filename_length-7)])
    temp = file.name.split('.')[0]
    filename_list.append(temp)
    # print(temp)

for idx in df_source.index:
    # print(str(df_source['vid_url'][idx])[-11:])
    if str(df_source['vid_url'][idx])[-11:] in filename_list:
        stance = df_source['normalized_annotation'][idx]
        if stance == 0:
            num_neutral += 1
        elif stance == 1:
            num_debunk += 1
        elif stance == -1:
            num_promote += 1

print("Neutral: " + str(num_neutral) + "\nPromote: " + str(num_promote) + "\nDebunk: " + str(num_debunk))

Neutral: 1536
Promote: 396
Debunk: 214


In [37]:
finished_df = pd.DataFrame(columns=['annotation', 'stance'])

In [54]:


for idx in df_source.index:
    if str(df_source['vid_url'][idx])[-11:] in filename_list:
        stance = df_source['normalized_annotation'][idx]

        if stance not in ['-1','0','1']:
            print('ISSUE ' + str(stance))
            break

        vtt_filename = str(df_source['vid_url'][idx])[-11:] + ".vtt"
        vtt_filename = 'subs/' + vtt_filename
        try:
            annotation = get_transcript(vtt_filename)
        except Exception as e:
            print(str(e))
        finished_df.loc[len(finished_df.index)] = [annotation, stance]

finished_df

    

ISSUE 0


,annotation,stance
0,ten years after 9/11 a new report out just th...,0
1,[Music] it happened outside Waco Texas a heav...,0
2,thanks for coming it's nice to see a good tur...,0
3,"&gt;&gt; Tonight... &gt;&gt; I, Donald John T...",0
4,"- This week on Buzzfeed Unsolved, we discuss ...",0
...,...,...
2667,dr. david groves is a physicist in the UK and...,1
2668,"- Hi, I'm Matt. And I don't believe we landed...",1
2669,,-1
2670,hi everybody what i'd like to talk about for ...,0


In [55]:
finished_df.iloc[5]

annotation     Hey everyone, my name is Ben and welcome on m...
stance                                                        0
Name: 5, dtype: object

In [57]:
finished_df.to_csv("finished_df_new.csv")

In [3]:
import pandas as pd
finished_df = pd.read_csv('finished_df.csv', index_col=0)

In [4]:
finished_df['stance'] = finished_df['stance'].apply(pd.to_numeric, errors = 'coerce').fillna(1)
finished_df['stance']

0       0
1       0
2       0
3       0
4       0
       ..
2667    1
2668    1
2669   -1
2670    0
2671   -1
Name: stance, Length: 2672, dtype: int64

In [5]:
finished_df['stance'] = finished_df['stance'] + 1

In [6]:
finished_df['annotation'] = finished_df['annotation'].astype('string')
finished_df = finished_df.dropna()
finished_df['annotation']

0        ten years after 9/11 a new report out just th...
1        [Music] it happened outside Waco Texas a heav...
2        thanks for coming it's nice to see a good tur...
3        &gt;&gt; Tonight... &gt;&gt; I, Donald John T...
4        - This week on Buzzfeed Unsolved, we discuss ...
                              ...                        
2666     astronauts moving in slow motion is another h...
2667     dr. david groves is a physicist in the UK and...
2668     - Hi, I'm Matt. And I don't believe we landed...
2670     hi everybody what i'd like to talk about for ...
2671     Apparently, there's an organization called "N...
Name: annotation, Length: 2664, dtype: string

In [7]:
finished_df

,annotation,stance
0,ten years after 9/11 a new report out just th...,1
1,[Music] it happened outside Waco Texas a heav...,1
2,thanks for coming it's nice to see a good tur...,1
3,"&gt;&gt; Tonight... &gt;&gt; I, Donald John T...",1
4,"- This week on Buzzfeed Unsolved, we discuss ...",1
...,...,...
2666,astronauts moving in slow motion is another h...,2
2667,dr. david groves is a physicist in the UK and...,2
2668,"- Hi, I'm Matt. And I don't believe we landed...",2
2670,hi everybody what i'd like to talk about for ...,1


In [11]:
# exploration
onehot = pd.get_dummies(finished_df['stance'])
onehot = onehot.rename({
    0: 'promote',
    1: 'neutral',
    2: 'debunk'
}, axis=1)
onehot

,promote,neutral,debunk
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
2666,False,False,True
2667,False,False,True
2668,False,False,True
2670,False,True,False


In [14]:
finished_df['promote'] = onehot['promote']
finished_df['neutral'] = onehot['neutral']
finished_df['debunk'] = onehot['debunk']
finished_df

,annotation,stance,promote,neutral,debunk
0,ten years after 9/11 a new report out just th...,1,False,True,False
1,[Music] it happened outside Waco Texas a heav...,1,False,True,False
2,thanks for coming it's nice to see a good tur...,1,False,True,False
3,"&gt;&gt; Tonight... &gt;&gt; I, Donald John T...",1,False,True,False
4,"- This week on Buzzfeed Unsolved, we discuss ...",1,False,True,False
...,...,...,...,...,...
2666,astronauts moving in slow motion is another h...,2,False,False,True
2667,dr. david groves is a physicist in the UK and...,2,False,False,True
2668,"- Hi, I'm Matt. And I don't believe we landed...",2,False,False,True
2670,hi everybody what i'd like to talk about for ...,1,False,True,False


In [16]:
y = onehot.values
y

array([[False,  True, False],
       [False,  True, False],
       [False,  True, False],
       ...,
       [False, False,  True],
       [False,  True, False],
       [ True, False, False]])

In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    finished_df['annotation'], y, test_size=0.15, shuffle=True
)

In [18]:
X_train

755      this is the story of the people who searched ...
301      [Music] China has long been a surveillance st...
42       this video is made possible by skill share le...
353      [Music] down this road on a summer day in 194...
1999     our mom is going tonight concerns breast impl...
                              ...                        
2127     I was going on today guys my name is Dwayne a...
1897     amid measles outbreaks and drops and vaccinat...
1695     I mean first off it's a relatively new vaccin...
1161     sometimes the truth is stranger than fiction ...
2641     five four three two one she's a another victi...
Name: annotation, Length: 2264, dtype: string

In [19]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [55]:
# bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# nn layers
# l1 = tf.keras.layers.Dropout(0.1)(outputs['pooled_output'])
l1 = tf.keras.layers.Dropout(0.1)(outputs['sequence_output'])
l2 = tf.keras.layers.LSTM(units=64)(l1)
l3 = tf.keras.layers.Dropout(0.1)(l2)
l4 = tf.keras.layers.Dense(units=3)(l3)

In [56]:
# CHANGE OUTPUT LAYER AFTER ADJUSTING 
model = tf.keras.Model(inputs = [text_input], outputs = [l4])

In [57]:
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['input_9[0][0]']                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [58]:
loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

In [59]:
model.compile(
    loss=loss,
    metrics=['accuracy'],
    optimizer='adam'
)

In [41]:
y_train = np.array(y_train).astype('int64')

In [42]:
y_train

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       ...,
       [1, 0, 0],
       [1, 0, 0],
       [0, 1, 0]], dtype=int64)

In [43]:
X_train

755      this is the story of the people who searched ...
301      [Music] China has long been a surveillance st...
42       this video is made possible by skill share le...
353      [Music] down this road on a summer day in 194...
1999     our mom is going tonight concerns breast impl...
                              ...                        
2127     I was going on today guys my name is Dwayne a...
1897     amid measles outbreaks and drops and vaccinat...
1695     I mean first off it's a relatively new vaccin...
1161     sometimes the truth is stranger than fiction ...
2641     five four three two one she's a another victi...
Name: annotation, Length: 2264, dtype: string

In [60]:
history = model.fit(X_train, y_train)

71/71 [==============================] - 175s 2s/step - loss: 0.6956 - accuracy: 0.7491
